In [1]:
import pandas as pd

In [2]:
file = open('./data/aclImdb/train/pos/0_9.txt')

In [126]:
import os
import sys
import time
import re
import string
import unicodecsv as csv
import unicodedata as un
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

in_path = './data/aclImdb/train/'
out_path= './data/aclImdb/train.csv'

In [130]:
counter = 0
error_counter = 0

# Start timer
start_time = time.time()
label = 0

# Prepare dictionary of necessary unicode
# Thanks to https://stackoverflow.com/a/11066687/4595807
# We want to protect '-'
# HYPHEN-MINUS = UNICODE DECIMAL VALUE = 45
table = dict.fromkeys(i for i in range(sys.maxunicode) 
                        if un.category(chr(i)).startswith(('P','N','S','Cf','Cn','Cc')))

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

stop_words = set(stopwords.words('english'))

header = 'label', 'data'

with open(out_path, 'wb') as out_file:
    writer = csv.writer(out_file, encoding='utf-8')
    writer.writerow(header)
    
    for root, dirs, files in os.walk(in_path, topdown=True):
        
        for name in files:
            curr_file = os.path.join(root, name)
            try:
                # Read current file and remove BOM and newline characters
                # uf03c and uf03c are invalid unicode characters and don't have any category
                # so had to remove manually
                fp = open(curr_file, encoding='utf-8-sig').read()
                
                # Remove html tags
                fp = fp.replace('<br />', '')
                
                # Remove all punctuations
                fp = fp.translate(translator)
                
                # Remove numbers/symbols/invalid chars
                fp = fp.translate(table)
                
                # Conver to lower case
                fp = fp.lower()
                
                # Tokenize the word
#                 fp = word_tokenize(fp)
                
                # Remove the stopwords
#                 fp = [w for w in fp if not w in stop_words]
                
                
                # Normalize the unicode so that
                # canonical-equivalent ones will also have precisely the same binary representation
                final_msg = label, fp
                
                # Write into CSV file format - label, data
                writer.writerow(final_msg)

                # Counter setup to count file processed
                counter = counter + 1
#                 break

            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))
                error_counter = error_counter + 1
#                 break
        label += 1
                

out_file.close()
    
end_time = time.time()

print('Number of files processed: ',counter)
print('Number of files error: ',error_counter)

print('Time taken in seconds:',(end_time - start_time))

Number of files processed:  25000
Number of files error:  0
Time taken in seconds: 8.349965572357178


In [131]:
pd_train = pd.read_csv('./data/aclImdb/train.csv')
pd_test = pd.read_csv('./data/aclImdb/test.csv')

In [132]:
pd_train.head()

,label,data
0,1,story of a man who has unnatural feelings for ...
1,1,airport starts as a brand new luxury plane ...
2,1,this film lacked something i couldn t put my f...
3,1,sorry everyone i know this is supposed to b...
4,1,when i was little my parents took me along to ...


In [133]:
train_token_count = 0
test_token_count = 0

for each_row in pd_train['data']:
    train_token_count += len(each_row)
    
for each_row in pd_test['data']:
    test_token_count += len(each_row)

print("Number of tokens in train = ", train_token_count)
print("Number of tokens in test = ", test_token_count)

Number of tokens in train =  32430009
Number of tokens in test =  28877902


In [134]:
print("Avg Number of tokens in each sentence in train = ", train_token_count/pd_train.shape[0])
print("Avg Number of tokens in each sentence in test = ", test_token_count/pd_test.shape[0])

Avg Number of tokens in each sentence in train =  1297.20036
Avg Number of tokens in each sentence in test =  1155.11608


In [182]:
from nltk.util import ngrams
from collections import Counter

tokens_list = pd_train.data.tolist()

In [185]:
tokens_list[10].split()[5]

'pant'

In [180]:
from nltk.util import ngrams
from collections import Counter

tokens_list = pd_train.data.tolist()

tokens = []
for each in tokens_list:
    for i in each.split():
        tokens.append(i)

In [179]:
text_trigrams = ngrams(tokens, 3)

Counter(text_trigrams).most_common(10)

[(('one', 'of', 'the'), 4941),
 (('i', 'don', 't'), 2705),
 (('this', 'movie', 'is'), 2674),
 (('of', 'the', 'film'), 2611),
 (('this', 'is', 'a'), 2379),
 (('it', 's', 'a'), 2376),
 (('a', 'lot', 'of'), 2276),
 (('of', 'the', 'movie'), 2182),
 (('some', 'of', 'the'), 1909),
 (('the', 'film', 'is'), 1872)]